In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import mne
from mne.viz import plot_topomap
import csv

In [2]:
def load_raw_data(data_path):
    #load the raw data
    raw = mne.io.read_raw_fif(data_path, preload=True)
    return raw

In [3]:
file_path = 'data/sub-CC310385/meg/passive_raw.fif'
raw = load_raw_data(file_path)

Opening raw data file /Users/mlion0200/GitRepo/eye_blink_detection/data/sub-CC310385/meg/passive_raw.fif...
    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle
    Range : 15000 ... 163999 =     15.000 ...   163.999 secs
Ready.
Current compensation grade : 0
Reading 0 ... 148999  =      0.000 ...   148.999 secs...


In [4]:
grad_pos = mne.find_layout(raw.info, ch_type='grad')

In [5]:
raw.info['chs'][0]

{'scanno': 1,
 'logno': 113,
 'kind': 1,
 'range': 1.9073486328125e-05,
 'cal': 3.250000046861601e-09,
 'coil_type': 3012,
 'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.0127    ,  0.0057    ,
        -0.99990302, -0.186801  , -0.98240298, -0.0033    , -0.98232698,
         0.18674099,  0.013541  ]),
 'unit': 201,
 'unit_mul': 0,
 'ch_name': 'MEG0113',
 'coord_frame': 1}

In [6]:
def get_channel_position(raw, ch_type):
    channel_pos_dict = {}
    grad_pos = mne.find_layout(raw.info, ch_type=ch_type)
    raw_grad = raw.copy().pick_types(meg='grad')
    for i in range(0, len(raw_grad.info['chs'])):
        channel_pos_dict[raw.info['chs'][i]['ch_name']] = raw.info['chs'][i]['loc'][:3].tolist()
    return channel_pos_dict

In [7]:
raw_grad = raw.copy().pick_types(meg='grad')
len(raw_grad.info['chs'])

204

In [8]:
channel_positions = get_channel_position(raw, 'grad')

In [9]:
channel_positions

{'MEG0113': [-0.10660000145435333, 0.04639999940991402, -0.06040000170469284],
 'MEG0112': [-0.10660000145435333, 0.04639999940991402, -0.06040000170469284],
 'MEG0111': [-0.10660000145435333, 0.04639999940991402, -0.06040000170469284],
 'MEG0122': [-0.10199999809265137, 0.06310000270605087, -0.025599999353289604],
 'MEG0123': [-0.10199999809265137, 0.06310000270605087, -0.025599999353289604],
 'MEG0121': [-0.10199999809265137, 0.06310000270605087, -0.025599999353289604],
 'MEG0132': [-0.10849999636411667, 0.03020000085234642, -0.026599999517202377],
 'MEG0133': [-0.10849999636411667, 0.03020000085234642, -0.026599999517202377],
 'MEG0131': [-0.10849999636411667, 0.03020000085234642, -0.026599999517202377],
 'MEG0143': [-0.10989999771118164, 0.013100000098347664, -0.06270000338554382],
 'MEG0142': [-0.10989999771118164, 0.013100000098347664, -0.06270000338554382],
 'MEG0141': [-0.10989999771118164, 0.013100000098347664, -0.06270000338554382],
 'MEG0213': [-0.10740000009536743, 0.032900

In [10]:
def preprocess_eog(eog):
    eog.filter(1, 15, method="iir", iir_params=None)
    return eog

In [11]:
def get_eog_channels(raw):
    picks = mne.pick_types(raw.info, meg=False, eeg=False, stim=False, eog=True, exclude='bads')
    raw.filter(1, 15, picks=picks, method="iir", iir_params=None)
    eog = raw.copy().pick_channels(['EOG062'])
    #raw = raw.pick_channels(['EOG061'])
    return eog

In [12]:
def get_eye_blink_time_points_array(eog):
    eog_data = eog.get_data()[0]
    threshold = np.max(eog_data)*2/3
    time_points = np.where(eog_data > threshold)
    return threshold, time_points[0]

In [13]:
def get_peak_time_point(time_points_array):
    start = time_points_array[0]
    end = time_points_array[0]
    peaks = []
    stored = time_points_array[0:5]
    for i in range(0, len(time_points_array)):
        if time_points_array[i] not in stored and (time_points_array[i]-1) not in stored:
            end = time_points_array[i-1]
            peak = start + int((end - start)/2)
            peaks.append(peak)
            start = time_points_array[i]
        if i >= 4:
            stored = time_points_array[i-4:i+1]
    return peaks

In [14]:
def random_markers(window_size, sampling_rate, marker):
    random_marker = []
    buffer = window_size*sampling_rate
    if len(marker) <= 0:
        return random_marker
    for i in range(len(marker)):
        if i == 0 and marker[0] > buffer:
            random_marker.append(random.randint(buffer,marker[0]))
        elif marker[i-1]+buffer < marker[i]-buffer:
            random_marker.append(random.randint(marker[i-1]+buffer,marker[i]-buffer))
    # HARD CODED length
    if marker[i]+buffer < 149000-buffer:
        random_marker.append(random.randint(marker[i]+buffer, 149000-buffer))
    return random_marker

In [15]:
def preprocessing(raw):
    picks = mne.pick_types(raw.info, meg='grad', eeg=False, stim=False, exclude='bads')
    #raw.filter(3, 40, picks=picks,l_trans_bandwidth='auto', filter_length='auto', phase='zero', fir_design='firwin')
    raw.filter(0.5, 25, picks=picks,l_trans_bandwidth='auto', filter_length='auto', phase='zero', fir_design='firwin')
    raw.notch_filter(60, picks=picks, fir_design='firwin')
    # channels = ['BG1-2511', 'BG2-2511', 'BG3-2511', 'BP1-2511', 'BP2-2511', 'BP3-2511', 'BQ1-2511', 'BQ2-2511', 'BQ3-2511', 'BR1-2511', 'BR2-2511', 'BR3-2511', 'G12-2511', 'G13-2511', 'G23-2511', 'P12-2511', 'P13-2511', 'Q12-2511', 'Q13-2511', 'Q21-2511', 'Q23-2511', 'R12-2511', 'R13-2511', 'R23-2511', 'SCLK01-177', 'G11-2511', 'G22-2511', 'P11-2511', 'P22-2511', 'Q11-2511', 'Q22-2511', 'R11-2511', 'R22-2511', 'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'EEG020', 'EEG021', 'EKG']
    # if b_drop_channels:
    # 	channels = ['BG1-2511', 'BG2-2511', 'BG3-2511', 'BP1-2511', 'BP2-2511', 'BP3-2511', 'BQ1-2511', 'BQ2-2511', 'BQ3-2511', 'BR1-2511', 'BR2-2511', 'BR3-2511', 'G12-2511', 'G13-2511', 'G23-2511', 'P12-2511', 'P13-2511', 'Q12-2511', 'Q13-2511', 'Q21-2511', 'Q23-2511', 'R12-2511', 'R13-2511', 'R23-2511', 'SCLK01-177', 'G11-2511', 'G22-2511', 'P11-2511', 'P22-2511', 'Q11-2511', 'Q22-2511', 'R11-2511', 'R22-2511', 'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2', 'EEG020', 'EEG021', 'EKG']
    # 	raw = raw.drop_channels(channels)
    return raw

In [16]:
def load_data_events(dir_name):
    file_path = os.path.join(os.getcwd(), 'data/sub-CC310385/meg/passive_raw.fif')
    raw =load_raw_data(file_path)
    data_preprocessed = preprocessing(raw)

    # add stimulus channel
    # positive = 1, negative = 2
    eog = get_eog_channels(raw)
    eye_blink_threshold, time_points_array = get_eye_blink_time_points_array(eog)
    eye_blink_peaks = get_peak_time_point(time_points_array)
    print("Eye blink peaks")
    print(eye_blink_peaks)
    non_eye_blink_peaks = random_markers(0.2, 1000, eye_blink_peaks)
    print("Non eye blink peaks")
    print(non_eye_blink_peaks)
    # sample need to between raw.first_samp and raw.last_samp
    
    eye_blinks = []
    labels = []
    for i in eye_blink_peaks:
        if i >= raw.first_samp and i <= raw.last_samp:
            eye_blink_raw = raw.copy().get_data().T[i]
            eye_blinks.append(eye_blink_raw)
            labels.append(1)
    for j in non_eye_blink_peaks:
        if j >= raw.first_samp and j <= raw.last_samp:
            non_eye_blink_raw = raw.copy().get_data().T[i]
            eye_blinks.append(non_eye_blink_raw)
            labels.append(2)
    return eye_blinks, labels

In [17]:
eye_blinks, labels = load_data_events('')

Opening raw data file /Users/mlion0200/GitRepo/eye_blink_detection/data/sub-CC310385/meg/passive_raw.fif...
    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle
    Range : 15000 ... 163999 =     15.000 ...   163.999 secs
Ready.
Current compensation grade : 0
Reading 0 ... 148999  =      0.000 ...   148.999 secs...
Filtering a subset of channels. The highpass and lowpass values in the measurement info will not be updated.
Setting up band-pass filter from 0.5 - 25 Hz
l_trans_bandwidth chosen to be 0.5 Hz
h_trans_bandwidth chosen to be 6.2 Hz

/anaconda3/envs/mne/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
